The purpose of this notebook is to create a CNN model for landmark detection. I copied all the code from the article we were originally planning to recreate. We don't necessarily have to use it.

Inputs:
- CSV of image information: /data/google-data/df_final.csv
- Images: all found in the folder /data/google-data/images
        
Outputs: unsure. Potentially may save the model if we decide to deploy this.

In [1]:
# Packages
import sys, multiprocessing, csv
import numpy as np
import pandas as pd
import keras
import cv2
from matplotlib import pyplot as plt
import os
import random
from urllib.request import urlopen
from io import BytesIO
from PIL import Image
from sklearn.preprocessing import LabelEncoder


# Deep Learning libraries
import tensorflow.keras as keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD

In [4]:
# Read in df
df = pd.read_csv("data/google-data/df_final.csv")

num_classes = len(df["landmark_id"].unique())

In [5]:
# Copying over some code from the analytics vidhya article/former notebook

# Parameters
learning_rate   = 0.0001
decay_speed     = 1e-6
momentum        = 0.09
loss_function   = "sparse_categorical_crossentropy"
source_model = VGG19(weights=None)
#new_layer = Dense(num_classes, activation=activations.softmax, name='prediction')
drop_layer = Dropout(0.5)
drop_layer2 = Dropout(0.5)
model = Sequential()
for layer in source_model.layers[:-1]: # go through until last layer
    if layer == source_model.layers[-25]:
        model.add(BatchNormalization())
    model.add(layer)
#     if layer == source_model.layers[-3]:
#         model.add(drop_layer)
# model.add(drop_layer2)
model.add(Dense(num_classes, activation="softmax"))
model.summary()
optim1 = keras.optimizers.RMSprop(learning_rate = 0.0001, momentum = 0.09)
optim2 = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
model.compile(optimizer=optim1,
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])
sgd = SGD(lr=learning_rate, decay=decay_speed, momentum=momentum, nesterov=True)
rms = keras.optimizers.RMSprop(learning_rate=learning_rate, momentum=momentum)
model.compile(optimizer=rms,
              loss=loss_function,
              metrics=["accuracy"])
print("Model compiled! n")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

/Users/margaretsabelhaus/opt/anaconda3/envs/dsci/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
#Function used to process the data, fitted into a data generator.

# Manually setting path
folder = '/Users/margaretsabelhaus/Documents/GitHub/CSCI575-Group12/data/google-data/images'

lencoder = LabelEncoder()
lencoder.fit(df["landmark_id"])
def encode_label(lbl):
    return lencoder.transform(lbl)
def decode_label(lbl):
    return lencoder.inverse_transform(lbl)
def get_image_from_number(num, df):
    #print(df.iloc[num,:])
    row = df.iloc[num,:]
    #print(row[0])
    #print(row[2])
    #fname, label = df.iloc[num,:]
    fname = row[0]
    label = row[2]
    fname = fname + ".jpg"
    path = os.path.join(folder,fname)
    im = cv2.imread(path)
    
    # // Error handling
    print(im.shape)
    # 2.28 spotted the issue: we're trying to read in images that do not exist.
    # The main DF references all images. However, we were unable to download some where the url was broken.
    # Not sure of any easy solutions.
    
    return im, label
def image_reshape(im, target_size):
    return cv2.resize(im, target_size)
def get_batch(dataframe,start, batch_size):
    image_array = []
    label_array = []
    end_img = start+batch_size
    if end_img > len(dataframe):
        end_img = len(dataframe)
    for idx in range(start, end_img):
        n = idx
        im, label = get_image_from_number(n, dataframe)
        im = image_reshape(im, (224, 224)) / 255.0
        image_array.append(im)
        label_array.append(label)
    label_array = encode_label(label_array)
    return np.array(image_array), np.array(label_array)
batch_size = 16
epoch_shuffle = True
weight_classes = True
epochs = 15

# Split train data up into 80% and 20% validation
train, validate = np.split(df.sample(frac=1), [int(.8*len(df))])
print("Training on:", len(train), "samples")
print("Validation on:", len(validate), "samples")
for e in range(epochs):
    print("Epoch: ", str(e+1) + "/" + str(epochs))
    if epoch_shuffle:
        train = train.sample(frac = 1)
    for it in range(int(np.ceil(len(train)/batch_size))):
        X_train, y_train = get_batch(train, it*batch_size, batch_size)
        
        # // Error handling
        #print(X_train)
        #print(y_train)
        # We seem to be able to read in the data fine
        
        model.train_on_batch(X_train, y_train)
#model.save("Model.h5")

Training on: 8282 samples
Validation on: 2071 samples
Epoch:  1/15
(1200, 1600, 3)
(342, 512, 3)
(1200, 1600, 3)
(1600, 1200, 3)
(1064, 1600, 3)
(1200, 1600, 3)
(1200, 1600, 3)
(768, 1024, 3)
(1067, 1600, 3)
(1024, 683, 3)
(960, 1280, 3)
(1200, 1600, 3)
(1200, 1600, 3)
(480, 640, 3)
(1200, 1600, 3)
(1069, 1600, 3)


2022-03-29 13:06:17.311344: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(1200, 1600, 3)
(1600, 1200, 3)
(1200, 1600, 3)
(1067, 1600, 3)
(1060, 1600, 3)
(340, 512, 3)
(1200, 1600, 3)
(384, 512, 3)
(1600, 1200, 3)
(1600, 1200, 3)
(1200, 1600, 3)
(1600, 1071, 3)
(1200, 1600, 3)
(1200, 1600, 3)
(1068, 1600, 3)
(1060, 1600, 3)
(1600, 1200, 3)
(1200, 1600, 3)
(1200, 1600, 3)
(336, 448, 3)
(1064, 1600, 3)
(800, 600, 3)
(1200, 1600, 3)
(1063, 1600, 3)
(1200, 1600, 3)
(1066, 1600, 3)
(1600, 1200, 3)
(384, 512, 3)
(1063, 1600, 3)
(1069, 1600, 3)
(1200, 1600, 3)
(1071, 1600, 3)
(1200, 1600, 3)
(1200, 1600, 3)
(1600, 1200, 3)
(480, 640, 3)
(1063, 1600, 3)
(1067, 1600, 3)
(1198, 1600, 3)
(1200, 1600, 3)
(1200, 1600, 3)
(1399, 1600, 3)
(1071, 1600, 3)
(1071, 1600, 3)
(1200, 1600, 3)
(1200, 1600, 3)
(768, 1024, 3)
(1600, 1200, 3)
